In [1]:
import os
import sys

In [2]:
import jax
print(jax.devices())          # expect GpuDevice on NVIDIA, MetalDevice on Mac
print(jax.default_backend())  # 'gpu' / 'metal' if accelerated


ERROR:2025-11-25 17:57:54,364:jax._src.xla_bridge:444: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 442, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 324, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 281, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


[CpuDevice(id=0)]
cpu


In [3]:
os.chdir('..')

# dbest-isi/searchless-chess-9M-selfplay
https://huggingface.co/dbest-isi/searchless-chess-9M-selfplay

In [4]:
!ls src

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


__pycache__	   distilled_student_model.pth	save_data_to_np.py
data_numpy.py	   eda.ipynb			searchless_chess_model
data_pytorch.py    jax_dataset.py		train_student.py
data_size.py	   lightning_logs		transformer_pytorch.py
dataset_loader.py  load_model.ipynb
distill_model.py   model_wrapper.py


In [5]:
from src.model_wrapper import create_predict_func
from huggingface_hub import snapshot_download

In [6]:
model_path = snapshot_download(
    repo_id="dbest-isi/searchless-chess-9M-selfplay",
    local_dir="./searchless_chess_model"
)
print(f"Model downloaded to: {model_path}")
model_path = '/teamspace/studios/this_studio/distill-chess/searchless_chess_model'

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Model downloaded to: /teamspace/studios/this_studio/distill-chess/searchless_chess_model


In [7]:
# Add bundled code to Python path
sys.path.insert(0, f"{model_path}/searchless_chess_code")

In [8]:

import hf_model
import utils
import tokenizer

# Load the model
model = hf_model.SearchlessChessModel.from_pretrained(model_path)
model.predict = create_predict_func(model, utils, tokenizer)

# Make a prediction
fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1"
result = model.predict(fen, temperature=1.0)

print(f"Best move: {result['best_move']}")
print(f"Q-value: {result['q_value']:.4f}")
print(f"Action probabilities shape: {result['action_probs'].shape}")

Best move: b7b5
Q-value: 0.7652
Action probabilities shape: (1968,)


In [9]:
print(model.params.keys())

dict_keys(['embed', 'embed_1', 'layer_norm', 'layer_norm_1', 'layer_norm_10', 'layer_norm_11', 'layer_norm_12', 'layer_norm_13', 'layer_norm_14', 'layer_norm_15', 'layer_norm_16', 'layer_norm_2', 'layer_norm_3', 'layer_norm_4', 'layer_norm_5', 'layer_norm_6', 'layer_norm_7', 'layer_norm_8', 'layer_norm_9', 'linear', 'linear_1', 'linear_10', 'linear_11', 'linear_12', 'linear_13', 'linear_14', 'linear_15', 'linear_16', 'linear_17', 'linear_18', 'linear_19', 'linear_2', 'linear_20', 'linear_21', 'linear_22', 'linear_23', 'linear_24', 'linear_3', 'linear_4', 'linear_5', 'linear_6', 'linear_7', 'linear_8', 'linear_9', 'multi_head_dot_product_attention/linear', 'multi_head_dot_product_attention/linear_1', 'multi_head_dot_product_attention/linear_2', 'multi_head_dot_product_attention/linear_3', 'multi_head_dot_product_attention_1/linear', 'multi_head_dot_product_attention_1/linear_1', 'multi_head_dot_product_attention_1/linear_2', 'multi_head_dot_product_attention_1/linear_3', 'multi_head_dot

# Run on dataset

In [10]:
!ls

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


 chess_transformer			 searchless_chess
 chessformer_epoch_13.pth		 searchless_chess_model
 chessformers				 src
 data					 train_student_model.py
 download_chessformer_model.sh		'view?usp=drive_link'
 plan_searchless_chess_distillation.md


In [11]:
from datasets import load_dataset, Dataset

# 1. Enable streaming
ds = load_dataset(
    "Lichess/chess-position-evaluations", 
    split="train[0:60_000_000]", # Download the first 60M samples
    # streaming=True
)

# 2. Filter for columns 'fen' and 'cp'
ds = ds.select_columns(["fen", "cp", 'mate'])

In [12]:
split_dataset = ds.train_test_split(test_size=0.2, seed=42)

In [13]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['fen', 'cp', 'mate'],
        num_rows: 48000000
    })
    test: Dataset({
        features: ['fen', 'cp', 'mate'],
        num_rows: 12000000
    })
})

In [14]:
!mkdir data/lichess_dataset

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


mkdir: cannot create directory ‘data/lichess_dataset’: File exists


In [15]:
# split_dataset['test'].to_parquet("data/lichess_dataset/test.parquet")
# split_dataset['train'].to_parquet("data/lichess_dataset/train.parquet")